## 自动求导

In [1]:
import torch
from torch.autograd import Variable

### 对标量进行自动求导

In [2]:
x = Variable(torch.Tensor([2]), requires_grad = True)
y = x + 2
z = y ** 2 + 3
z.backward()
print(x.grad)

tensor([8.])


更复杂的例子

In [3]:
x = Variable(torch.randn(10, 10), requires_grad=True)
y = Variable(torch.randn(10, 5), requires_grad=True)
w = Variable(torch.randn(10, 5), requires_grad=True)
out = torch.mean(y - torch.matmul(x, w)) # torch.matmul 矩阵乘法
out.backward()
print(x.grad)
print(y.grad)
print(w.grad)

tensor([[-0.0334,  0.0303,  0.0692, -0.0453, -0.0485,  0.0187, -0.0848,  0.0009,
          0.0673,  0.0100],
        [-0.0334,  0.0303,  0.0692, -0.0453, -0.0485,  0.0187, -0.0848,  0.0009,
          0.0673,  0.0100],
        [-0.0334,  0.0303,  0.0692, -0.0453, -0.0485,  0.0187, -0.0848,  0.0009,
          0.0673,  0.0100],
        [-0.0334,  0.0303,  0.0692, -0.0453, -0.0485,  0.0187, -0.0848,  0.0009,
          0.0673,  0.0100],
        [-0.0334,  0.0303,  0.0692, -0.0453, -0.0485,  0.0187, -0.0848,  0.0009,
          0.0673,  0.0100],
        [-0.0334,  0.0303,  0.0692, -0.0453, -0.0485,  0.0187, -0.0848,  0.0009,
          0.0673,  0.0100],
        [-0.0334,  0.0303,  0.0692, -0.0453, -0.0485,  0.0187, -0.0848,  0.0009,
          0.0673,  0.0100],
        [-0.0334,  0.0303,  0.0692, -0.0453, -0.0485,  0.0187, -0.0848,  0.0009,
          0.0673,  0.0100],
        [-0.0334,  0.0303,  0.0692, -0.0453, -0.0485,  0.0187, -0.0848,  0.0009,
          0.0673,  0.0100],
        [-0.0334,  

### 复杂情况的自动求导
对多维数组的自动求导机制

In [4]:
m = Variable(torch.FloatTensor([[2, 3]]), requires_grad=True) # 构建一个 1 x 2 的矩阵 数据分别是 2 ，3
n = Variable(torch.zeros(1, 2)) # 构建一个相同大小的 0 矩阵
print(m)
print(n)

tensor([[2., 3.]], requires_grad=True)
tensor([[0., 0.]])


In [5]:
# 通过 m 中的值计算新的 n 中的值
n[0, 0] = m[0, 0] ** 2
n[0, 1] = m[0, 1] ** 3
print(n)

tensor([[ 4., 27.]], grad_fn=<CopySlices>)


在 PyTorch 中，多维如果要调用自动求导，需要往backward()中传入一个参数，这个参数的形状和 n 一样大，比如是 $(w_0,\ w_1)$，那么自动求导的结果就是：
$$
\frac{\partial n}{\partial m_0} = w_0 \frac{\partial n_0}{\partial m_0} + w_1 \frac{\partial n_1}{\partial m_0}
$$$$
\frac{\partial n}{\partial m_1} = w_0 \frac{\partial n_0}{\partial m_1} + w_1 \frac{\partial n_1}{\partial m_1}
$$

In [6]:
n.backward(torch.ones_like(n))
print(m.grad)

tensor([[ 4., 27.]])


## 多次自动求导
调用 backward 我们可以进行一次自动求导，如果我们再调用一次 backward，程序会报错，没有办法再做一次。这是因为 PyTorch 默认做完一次自动求导之后，计算图就被丢弃。所以两次自动求导需要手动设置。eg.

In [7]:
x = Variable(torch.FloatTensor([3]), requires_grad=True)
y = x * 2 + x ** 2 + 3
y.backward(retain_graph = True) # 设置 retain_graph 为 True 来保留计算图
print(x.grad.data)

tensor([8.])


In [8]:
y.backward() # 再做一次自动求导，这次不保留计算图
print(x.grad)

tensor([16.])


x 的梯度变成了 16，因为这里做了两次自动求导，所以第一次的梯度 8 和第二次的梯度 8 加起来得到了 16 的结果

小练习

定义

$$
x = 
\left[
\begin{matrix}
x_0 \\
x_1
\end{matrix}
\right] = 
\left[
\begin{matrix}
2 \\
3
\end{matrix}
\right]
$$$$
k = (k_0,\ k_1) = (x_0^2 + 3 x_1,\ 2 x_0 + x_1^2)
$$
我们希望求得

$$
j = \left[
\begin{matrix}
\frac{\partial k_0}{\partial x_0} & \frac{\partial k_0}{\partial x_1} \\
\frac{\partial k_1}{\partial x_0} & \frac{\partial k_1}{\partial x_1}
\end{matrix}
\right]
$$
参考答案：

$$
\left[
\begin{matrix}
4 & 3 \\
2 & 6 \\
\end{matrix}
\right]
$$

In [9]:
x = Variable(torch.FloatTensor([2,3]),requires_grad = True )
k = Variable(torch.zeros(2))
k[0] = x[0] ** 2 + x[1] * 3 
k[1] = x[1] ** 2 + x[0] * 2 
print(k)

tensor([13., 13.], grad_fn=<CopySlices>)


In [10]:
j = torch.zeros(2,2)
k.backward(torch.FloatTensor([1, 0]), retain_graph=True)
j[0] = x.grad

x.grad.data.zero_() # 归零之前求得的梯度

k.backward(torch.FloatTensor([0, 1]))
j[1] = x.grad
print(j)

tensor([[4., 3.],
        [2., 6.]])
